In [1]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
    convert_var_to_povm
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)



In [2]:
from quara.data_analysis import data_analysis, physicality_violation_check

In [3]:
%load_ext autoreload
%autoreload 2

# MSE

In [4]:
def calc_estimate(
    tester_objects: List[State],
    true_object: Povm,
    measurement_n: int,
    num_data: List[int],
    iteration: int,
    estimator=StandardQTomographyEstimator,
    on_para_eq_constraint: bool = True,
) -> StandardQTomographyEstimationResult:

    povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    results = []
    for ite in tqdm(range(iteration)):
        empi_dists_seq = povmt.generate_empi_dists_sequence(true_object, num_data)
        result = estimator.calc_estimate_sequence(
            qtomography=povmt, empi_dists_sequence=empi_dists_seq, is_computation_time_required=True
        )

        # info = {
        #    "iteration": ite + 1,
        #     "data": empi_dists_seq,
        #     "estimated_var_sequence": result.estimated_var_sequence,
        #     "computation_times": result.computation_times,
        # }
        # print(info)
        results.append(result)

    identify = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    # with open(f"povmt_data/result_{identify}.pkl", "wb") as f:
    #    pickle.dump(results, f)
    return results

In [5]:
def calc_estimate_for_debug(
    tester_objects: List[State],
    true_object: Povm,
    measurement_n: int,
    num_data: List[int],
    iteration: int,
    estimator=StandardQTomographyEstimator,
    on_para_eq_constraint: bool = True,
) -> StandardQTomographyEstimationResult:

    povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    results = []
    for ite in tqdm(range(iteration)):
        # empi_dists_seq = povmt.generate_empi_dists_sequence(true_object, num_data)
        true_prob_dists = povmt.generate_prob_dists_sequence(true_object, num_data)
        true_prob_dists_seq = []
        for num in num_data:
            tmp_prob_dists = []
            for prob_dist in true_prob_dists:
                tmp_prob_dists.append((num, prob_dist))
            true_prob_dists_seq.append(tmp_prob_dists)
        
        # print(true_prob_dists_seq)
        # print("---------------------")
        
        # result = estimator.calc_estimate_sequence(
        #     qtomography=povmt, empi_dists_sequence=empi_dists_seq, is_computation_time_required=True
        # )
        result = estimator.calc_estimate_sequence(
             qtomography=povmt, empi_dists_sequence=true_prob_dists_seq, is_computation_time_required=True
        )

        # info = {
        #    "iteration": ite + 1,
        #     "data": empi_dists_seq,
        #     "estimated_var_sequence": result.estimated_var_sequence,
        #     "computation_times": result.computation_times,
        # }
        # print(info)
        results.append(result)

    identify = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    # with open(f"povmt_data/result_{identify}.pkl", "wb") as f:
        # pickle.dump(results, f)
    return results, povmt

# Case 1 (r = 1)

In [6]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
states = [state_x0, state_y0, state_z0, state_z1]

In [7]:
a0, a1, a2, a3 = 1, 1/np.sqrt(2), 0, 1/np.sqrt(2)

m1 = (1/np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1/np.sqrt(2)) * np.array([2-a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

In [22]:
# parameter
num_data = [100, 1000, 10000, 100000]
iterations = 100
measurement_n = len(true_object.vecs)  # 測定値の数

estimator = LinearEstimator()
proj_estimator = ProjectedLinearEstimator()

start = time.time()

param_affine_est_linear, povmt = calc_estimate_for_debug(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear, povmt = calc_estimate_for_debug(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)
p_param_affine_est_linear, povmt = calc_estimate_for_debug(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=True)
p_param_linear_est_linear, povmt = calc_estimate_for_debug(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=False)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time/60) + "[min]")

100%|██████████| 100/100 [00:00<00:00, 113.09it/s]

elapsed_time:0.03763781388600667[min]


In [23]:
p_param_linear_est_linear, povmt = calc_estimate_for_debug(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=False)

100%|██████████| 100/100 [00:00<00:00, 104.07it/s]


In [24]:
mses_affine, stds_affine, comp_time_affine = data_analysis.convert_to_series(param_affine_est_linear, true_object)

In [25]:
print("on_para_eq_constraint=True")
data_analysis.show_mse(num_data, mses_affine)

on_para_eq_constraint=True


In [26]:
mses_affine

[3.511451771104028e-30,
 3.511451771104028e-30,
 3.511451771104028e-30,
 3.511451771104028e-30]

In [27]:
print("on_para_eq_constraint=False")
mses_linear, stds_linear, comp_time_linear = data_analysis.convert_to_series(param_linear_est_linear, true_object)

data_analysis.show_mse(num_data, mses_linear)

on_para_eq_constraint=False


In [28]:
mses_pj_true, stds_pj_true, _ = data_analysis.convert_to_series(p_param_affine_est_linear, true_object)
mses_pj_false, stds_pj_false, _ = data_analysis.convert_to_series(p_param_linear_est_linear, true_object)

print(f"r=1")
title = f"Mean Square Value<br>Nrep={iterations}"
mses_list = [mses_affine, mses_linear, mses_pj_true, mses_pj_false]
name_list = ["LinearEstimator(True)", "LinearEstimator(False)", "ProjectedLinearEstimator(True)", "ProjectedLinearEstimator(False)"]

data_analysis.show_mses(num_data=num_data, mses=mses_list,
          names=name_list, title=title)

r=1


In [29]:
start = time.time()

param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True) 
param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)
p_param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=True)
p_param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=False)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time/60) + "[min]")

100%|██████████| 100/100 [01:52<00:00,  1.13s/it]

elapsed_time:7.446468432744344[min]


In [30]:
mses_affine, stds_affine, comp_time_affine = data_analysis.convert_to_series(param_affine_est_linear, true_object)
mses_linear, stds_linear, comp_time_linear = data_analysis.convert_to_series(param_linear_est_linear, true_object)
mses_pj_true, stds_pj_true, _ = data_analysis.convert_to_series(p_param_affine_est_linear, true_object)
mses_pj_false, stds_pj_false, _ = data_analysis.convert_to_series(p_param_linear_est_linear, true_object)

print(f"r=1")
title = f"Mean Square Value<br>Nrep={iterations}"
mses_list = [mses_affine, mses_linear, mses_pj_true, mses_pj_false]
name_list = ["LinearEstimator(True)", "LinearEstimator(False)", "ProjectedLinearEstimator(True)", "ProjectedLinearEstimator(False)"]

data_analysis.show_mses(num_data=num_data, mses=mses_list,
          names=name_list, title=title)

r=1


# Case 2 (r = 0.95)

In [17]:
# parameter
num_data = [100, 1000, 10000, 100000]
iterations = 100
measurement_n = len(true_object.vecs)  # 測定値の数

a0, a1, a2, a3 = 1, 1/np.sqrt(2), 0, 1/np.sqrt(2)
r = 0.95
a0, a1, a2, a3 = a0, r * a1, r * a2, r * a3

m1 = (1/np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1/np.sqrt(2)) * np.array([2-a0, -a1, -a2, -a3])
true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

estimator = LinearEstimator()

param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)

100%|██████████| 100/100 [02:47<00:00,  1.67s/it]


In [26]:
mses_affine, stds_affine, comp_time_affine = data_analysis.convert_to_series(param_affine_est_linear, true_object)
mses_linear, stds_linear, comp_time_linear = data_analysis.convert_to_series(param_linear_est_linear, true_object)

In [27]:
print(f"on_para_eq_constraint=True, r={r}")
data_analysis.show_mse(num_data, mses_affine)

on_para_eq_constraint=True, r=0.95


In [28]:
print(f"on_para_eq_constraint=True, r={r}")
data_analysis.show_mse(num_data, mses_linear)

on_para_eq_constraint=True, r=0.95


In [29]:
print(f"r={r}")
data_analysis.show_mses(num_data=num_data, mses=[mses_affine, mses_linear],
          names=["on_para_eq_constraint=True", "on_para_eq_constraint=False"])

r=0.95
